In [ ]:
# Analysis: parse logs and results, create tables and plots, save to results folder
import json
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
results_dir = Path("results")
results_dir.mkdir(parents=True, exist_ok=True)
log_path = results_dir.parent.joinpath("rfdetr_results", 'log.txt') if (Path('rfdetr_results')/ 'log.txt').exists() else Path('rfdetr_results')/ 'log.txt'
# prefer local top-level results.json if present, otherwise rfdetr_results/results.json
results_json_path = Path('results.json') if Path('results.json').exists() else (Path('rfdetr_results')/ 'results.json')
print('Analysis will write into:', results_dir)
print('Using log path:', log_path)
print('Using results.json path:', results_json_path)

In [ ]:
# Read and normalize epoch logs from rfdetr_results/log.txt and write CSVs and plots to results/
logs = []
if log_path.exists():
    with open(log_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                logs.append(obj)
            except Exception:
                # skip malformed lines
                continue

if logs:
    df_epochs = pd.json_normalize(logs)
    if 'epoch' in df_epochs.columns:
        df_epochs = df_epochs.sort_values('epoch').reset_index(drop=True)
    (results_dir / 'metrics_by_epoch.csv').write_text(df_epochs.to_csv(index=False))
    print('Wrote metrics_by_epoch.csv with', len(df_epochs), 'rows')
else:
    df_epochs = pd.DataFrame()
    print('No epoch logs found at', log_path)

# Read results.json (prefer top-level then rfdetr_results) and save per-class tables if available
rj = None
if results_json_path.exists():
    with open(results_json_path, 'r', encoding='utf-8') as f:
        try:
            rj = json.load(f)
        except Exception as e:
            print('Failed to parse results.json:', e)

def extract_and_save_classmap(obj, name):
    if not obj:
        return
    try:
        df = pd.DataFrame(obj)
        (results_dir / f'class_metrics_{name}.csv').write_text(df.to_csv(index=False))
        print(f'Wrote class_metrics_{name}.csv with {len(df)} rows')
    except Exception as e:
        print('Failed to write class map for', name, e)

if rj is not None:
    # handle common structures
    if 'class_map' in rj and isinstance(rj['class_map'], dict):
        for split, cm in rj['class_map'].items():
            extract_and_save_classmap(cm, split)
    for split_name in ('valid','test','train'):
        if split_name in rj:
            val = rj[split_name]
            if isinstance(val, dict) and 'class_map' in val:
                extract_and_save_classmap(val['class_map'], split_name)
            elif isinstance(val, list):
                extract_and_save_classmap(val, split_name)
    # Save full results json into results/ for reproducibility
    with open(results_dir / 'results_full.json', 'w', encoding='utf-8') as f:
        json.dump(rj, f, indent=2)
    print('Wrote results_full.json')
else:
    print('No results.json found at', results_json_path)

# Plots: loss, mAP, and per-class map (when data available)
if not df_epochs.empty:
    plt.figure(figsize=(8,5))
    if 'train_loss' in df_epochs.columns:
        plt.plot(df_epochs['epoch'], df_epochs['train_loss'], marker='o', label='train_loss')
    if 'test_loss' in df_epochs.columns:
        plt.plot(df_epochs['epoch'], df_epochs['test_loss'], marker='o', label='test_loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss per epoch')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(results_dir / 'loss_per_epoch.png')
    plt.close()
    print('Saved loss_per_epoch.png')

    # mAP over epochs: try multiple keys
    map_series = None
    if 'test_results_json.map' in df_epochs.columns:
        map_series = pd.to_numeric(df_epochs['test_results_json.map'], errors='coerce')
    elif 'test_coco_eval_bbox' in df_epochs.columns:
        try:
            map_series = df_epochs['test_coco_eval_bbox'].apply(lambda x: x[0] if isinstance(x, list) and len(x)>0 else None)
        except Exception:
            map_series = None
    if map_series is not None and map_series.dropna().size>0:
        plt.figure(figsize=(8,5))
        plt.plot(df_epochs['epoch'], map_series, marker='o')
        plt.xlabel('epoch')
        plt.ylabel('mAP')
        plt.title('mAP over epochs')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(results_dir / 'map_over_epochs.png')
        plt.close()
        print('Saved map_over_epochs.png')

# Per-class barplot from results.json (valid split preferred)
cm_valid = None
if rj is not None:
    if isinstance(rj, dict) and 'class_map' in rj and isinstance(rj['class_map'], dict) and 'valid' in rj['class_map']:
        cm_valid = rj['class_map']['valid']
    elif 'valid' in rj and isinstance(rj['valid'], list):
        cm_valid = rj['valid']
    elif 'valid' in rj and isinstance(rj['valid'], dict) and 'class_map' in rj['valid']:
        cm_valid = rj['valid']['class_map']
    elif 'test' in rj and isinstance(rj['test'], list):
        cm_valid = rj['test']

if cm_valid:
    df_cm = pd.DataFrame(cm_valid)
    map_col = None
    for c in ('map@50:95','map@50','map'):
        if c in df_cm.columns:
            map_col = c
            break
    if map_col is not None:
        df_cm = df_cm.sort_values(map_col, ascending=False)
        (results_dir / 'class_metrics_valid.csv').write_text(df_cm.to_csv(index=False))
        plt.figure(figsize=(10,6))
        sns.barplot(x=map_col, y='class', data=df_cm)
        plt.xlabel(map_col)
        plt.title('Per-class mAP (valid/test)')
        plt.tight_layout()
        plt.savefig(results_dir / 'per_class_map_valid.png')
        plt.close()
        print('Saved per_class_map_valid.png and class_metrics_valid.csv')

print('Analysis complete — CSVs and plots are in', results_dir)